In [46]:
#--- get last rows of feature maps
# assume that we have (9x9) x (7x7) patches 

import numpy as np 

mat = np.random.randn(2,9,9,7,7)

In [47]:
mat.shape

(2, 9, 9, 7, 7)

In [49]:
# --- 
mat[:,8,:,:,:].shape


(2, 9, 7, 7)

In [89]:
x = np.arange(10)
x.shape = (10)
x

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [102]:
x[-4:-2]

array([6, 7])

In [82]:
x[-5:-2]

array([15, 16, 17])

In [59]:
# --- 
mat[:,8,:,:,:]

array([[[[ 4.16658294e-01, -1.33213743e+00, -1.85143931e+00,
           2.16238958e-01, -1.32948263e+00, -1.33669568e+00,
           5.20154929e-01],
         [-8.18489093e-01,  1.20145815e+00,  1.76497192e+00,
          -4.13187798e-01, -9.77696275e-01, -3.62663393e-01,
           1.96148179e+00],
         [ 6.12138475e-01,  3.10523373e-01, -3.37443180e-01,
           1.10391871e+00, -6.35337778e-01, -6.98134612e-01,
           4.65438919e-01],
         [-1.35655296e+00,  7.63490660e-01, -2.00408786e-01,
          -1.55850586e+00,  2.25078532e+00,  4.37969743e-01,
           4.62123760e-01],
         [-4.59344301e-01,  4.79140674e-02, -4.40599137e-01,
           5.15250578e-01,  4.72873420e-02, -1.08449819e+00,
           9.98729005e-01],
         [ 5.84435890e-01, -9.60003145e-01, -5.25878126e-01,
           4.61502851e-01, -1.21912178e-01,  1.50879882e+00,
           6.35824315e-01],
         [-2.09330781e+00, -1.45539595e+00, -1.13473006e-01,
          -1.05137008e+00,  3.48740860e

In [5]:
#----
# ref : https://github.com/berniwal/swin-transformer-pytorch/blob/master/example.py
#---
import torch
from swin_transformer_pytorch import SwinTransformer

net = SwinTransformer(
    hidden_dim=96,
    layers=(2, 2, 6, 2),
    heads=(3, 6, 12, 24),
    channels=3,
    num_classes=3,
    head_dim=32,
    window_size=7,
    downscaling_factors=(4, 2, 2, 2),
    relative_pos_embedding=True
)
dummy_x = torch.randn(1, 3, 224, 224)
logits = net(dummy_x)  # (1,3)
print(net)
print(logits)

ModuleNotFoundError: No module named 'swin_transformer_pytorch'

In [22]:
import math
x = dummy_x = torch.randn(1, 3, 224, 224)
height = width = int(math.sqrt(x.shape[2])/2)

In [27]:
math.sqrt(224)/2

7.483314773547883

In [25]:
width

7

In [6]:
import torch
from torch import nn, einsum
import numpy as np
from einops import rearrange, repeat


class CyclicShift(nn.Module):
    def __init__(self, displacement):
        super().__init__()
        self.displacement = displacement

    def forward(self, x):
        return torch.roll(x, shifts=(self.displacement, self.displacement), dims=(1, 2))


In [9]:
def create_mask(window_size, displacement, upper_lower, left_right):
    mask = torch.zeros(window_size ** 2, window_size ** 2)

    if upper_lower:
        mask[-displacement * window_size:, :-displacement * window_size] = float('-inf')
        mask[:-displacement * window_size, -displacement * window_size:] = float('-inf')

    if left_right:
        mask = rearrange(mask, '(h1 w1) (h2 w2) -> h1 w1 h2 w2', h1=window_size, h2=window_size)
        mask[:, -displacement:, :, :-displacement] = float('-inf')
        mask[:, :-displacement, :, -displacement:] = float('-inf')
        mask = rearrange(mask, 'h1 w1 h2 w2 -> (h1 w1) (h2 w2)')

    return mask

In [8]:
window_size = 7 
displacement = 3

In [10]:
mask = torch.zeros(window_size**2,window_size**2 )

In [12]:
mask

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [14]:
mask[-displacement * window_size:, :-displacement * window_size] = float('-inf')
mask[:-displacement * window_size, -displacement * window_size:] = float('-inf')

In [15]:
mask

tensor([[0., 0., 0.,  ..., -inf, -inf, -inf],
        [0., 0., 0.,  ..., -inf, -inf, -inf],
        [0., 0., 0.,  ..., -inf, -inf, -inf],
        ...,
        [-inf, -inf, -inf,  ..., 0., 0., 0.],
        [-inf, -inf, -inf,  ..., 0., 0., 0.],
        [-inf, -inf, -inf,  ..., 0., 0., 0.]])

25

In [ ]:
#--

window_size = 7 
displacement = 3 

#---
displacement = window_size // 2
self.cyclic_shift = CyclicShift(-displacement)
self.cyclic_back_shift = CyclicShift(displacement)
self.upper_lower_mask = nn.Parameter(create_mask(window_size=window_size, displacement=displacement,
                                                    upper_lower=True, left_right=False), requires_grad=False)
self.left_right_mask = nn.Parameter(create_mask(window_size=window_size, displacement=displacement,